# train,test 셋 분류하기

In [28]:
import pandas as pd
import missingno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.max_columns = 999

In [29]:
plant = pd.read_csv('data/21.savedata_plant/pre_palnt.csv')

In [30]:
test = pd.read_csv('data/21.savedata_plant/pre_test.csv')

In [31]:
test_data = pd.read_csv("data/31.finaldata_plant/test_data.csv")

## plant train 분류

In [32]:
timeList = ['MEA_DDHR', 'X24H_TMA', 'X48H_TMA']
plant = plant.drop(timeList, axis=1)
plant.head()

,X24H_COND_LOC,X48H_COND_LOC,TEM_IN,HUM_IN,TEM_COIL,TEM_OUT_LOC1,HUM_OUT_LOC1,DEW_POINT_IN,PLANT_1,PLANT_2,LOC_1,LOC_2,LOC_3
0,0.0,0.0,-0.036700,-2.057425,-0.486644,-0.416944,-1.060988,-0.948895,1,0,1,0,0
1,0.0,0.0,-0.237239,-1.768877,-0.585582,-0.597441,-0.155766,-0.919136,1,0,1,0,0
2,0.0,0.0,-0.337508,-1.408192,-0.585582,-0.687689,-0.315511,-0.802814,1,0,1,0,0
3,0.0,0.0,-0.337508,-1.408192,-0.585582,0.395291,-1.699968,-0.802814,1,0,1,0,0
4,0.0,0.0,-0.036700,-1.768877,-0.585582,0.395291,-2.232452,-0.768177,1,0,1,0,0


In [33]:
train_X = plant.drop(['X24H_COND_LOC', 'X48H_COND_LOC'], axis=1)
train24_y = plant[['X24H_COND_LOC']]
train48_y = plant[['X48H_COND_LOC']]

## SMOTE를 활용한 오버샘플링
### 24시간 후 데이터 오버 샘플링

In [34]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled24, y_resampled24 = smote.fit_sample(train_X, train24_y)

In [35]:
X_resampled24.shape

(694676, 11)

In [36]:
from sklearn.model_selection import train_test_split
X_train24, X_val24, y_train24, y_val24 = train_test_split(X_resampled24,y_resampled24, test_size=0.3, shuffle=True, random_state=42)


In [37]:
X_train24.shape, X_val24.shape, y_train24.shape, y_val24.shape

((486273, 11), (208403, 11), (486273, 1), (208403, 1))

### 48시간 후 데이터 오버 샘플링

In [38]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled48, y_resampled48 = smote.fit_sample(train_X, train48_y)

In [39]:
X_resampled48.shape

(694676, 11)

In [40]:
from sklearn.model_selection import train_test_split
X_train48, X_val48, y_train48, y_val48 = train_test_split(X_resampled48,y_resampled48, test_size=0.3, shuffle=True, random_state=42)


In [41]:
X_train48.shape, X_val48.shape, y_train48.shape, y_val48.shape

((486273, 11), (208403, 11), (486273, 1), (208403, 1))

## test_data 만들기

In [42]:
timeList = ['MEA_DDHR', 'X24H_TMA', 'X48H_TMA']
test = test.drop(timeList, axis=1)
test_X = test.drop(['X24H_COND_LOC', 'X48H_COND_LOC'], axis=1)
test24_y = test[['X24H_COND_LOC']]
test48_y = test[['X48H_COND_LOC']]

## Valiation 함수 만들기

In [43]:
def makeValiation(model24,model48):
    """ 검증 데이터를 만들기 위한 함수입니다. 24후 모델,48후 모델값을 넣어 주어야 합니다."""
    validation = test_data[["MEA_DDHR","PLANT", "LOC", "X24H_TMA", "X48H_TMA"]]
    # 24시간 후 예측하기, 확률로 나누기
    predict24 = model24.predict(test_X)
    predict24_proba = model24.predict_proba(test_X)
    # 48시간 후 예측하기, 확률로 나누기
    predict48 = model48.predict(test_X)
    predict48_proba  = model48.predict_proba(test_X)
    # X24H_COND_LOC, X24H_COND_LOC_PROB, X48H_COND_LOC, X48H_COND_LOC_PROB
    validation["X24H_COND_LOC"] = predict24.astype(int)
    validation["X24H_COND_LOC_PROB"] = (predict24_proba[:,1]*100).astype(int)
    validation["X48H_COND_LOC"] = (predict48).astype(int)
    validation["X48H_COND_LOC_PROB"] = (predict48_proba[:,1]*100).astype(int)
    # 검증 형식 맞춰주기
    validation = validation[["MEA_DDHR","PLANT", "LOC", "X24H_TMA","X24H_COND_LOC", "X24H_COND_LOC_PROB", "X48H_TMA", "X48H_COND_LOC", "X48H_COND_LOC_PROB"]]
    return validation

# Random Forest
## 24시간 후 결로 예측

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf_model24 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model24.fit(X_train24, y_train24)

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [45]:
y_pred24 = rf_model24.predict(X_val24)
from sklearn.metrics import classification_report
print(classification_report(y_val24, y_pred24))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    104169
         1.0       1.00      1.00      1.00    104234

    accuracy                           1.00    208403
   macro avg       1.00      1.00      1.00    208403
weighted avg       1.00      1.00      1.00    208403



## 48시간 결로 예측

In [46]:
from sklearn.ensemble import RandomForestClassifier
rf_model48 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model48.fit(X_train48, y_train48)

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [47]:
y_pred48 = rf_model48.predict(X_val48)
from sklearn.metrics import classification_report
print(classification_report(y_val48, y_pred48))

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00    104163
         1.0       0.99      1.00      1.00    104240

    accuracy                           1.00    208403
   macro avg       1.00      1.00      1.00    208403
weighted avg       1.00      1.00      1.00    208403



## 검증값 만들기

In [48]:
validation = makeValiation(rf_model24, rf_model48)
validation

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 00:00:00,2,3,2019-04-02 00:00:00,0,0,2019-04-03 00:00:00,0,1
1,2019-04-01 04:30:00,1,2,2019-04-02 04:30:00,0,0,2019-04-03 04:30:00,0,0
2,2019-04-01 10:30:00,2,3,2019-04-02 10:30:00,0,0,2019-04-03 10:30:00,0,0
3,2019-04-01 11:00:00,1,1,2019-04-02 11:00:00,0,1,2019-04-03 11:00:00,0,0
4,2019-04-01 13:30:00,2,1,2019-04-02 13:30:00,0,0,2019-04-03 13:30:00,0,0
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 03:00:00,1,2,2020-03-30 03:00:00,0,0,2020-03-31 03:00:00,0,0
3535,2020-03-29 05:00:00,2,3,2020-03-30 05:00:00,0,0,2020-03-31 05:00:00,0,0
3536,2020-03-29 07:00:00,2,1,2020-03-30 07:00:00,0,5,2020-03-31 07:00:00,0,2
3537,2020-03-29 11:30:00,1,1,2020-03-30 11:30:00,0,0,2020-03-31 11:30:00,0,0


In [49]:
def findCondRow24(validation):
    data = validation.loc[validation["X24H_COND_LOC"] == 1]
    return data
def findCondRow48(validation):
    data = validation.loc[validation["X48H_COND_LOC"] == 1]
    return data    

In [50]:
findCondRow24(validation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
421,2019-05-15 12:30:00,2,1,2019-05-16 12:30:00,1,51,2019-05-17 12:30:00,0,0
1913,2019-11-11 14:30:00,1,2,2019-11-12 14:30:00,1,64,2019-11-13 14:30:00,0,12
1914,2019-11-11 18:00:00,1,3,2019-11-12 18:00:00,1,64,2019-11-13 18:00:00,0,1
2044,2019-11-21 07:30:00,1,3,2019-11-22 07:30:00,1,56,2019-11-23 07:30:00,0,0
2072,2019-11-22 21:00:00,1,2,2019-11-23 21:00:00,1,72,2019-11-24 21:00:00,0,20
2094,2019-11-24 03:00:00,1,3,2019-11-25 03:00:00,1,71,2019-11-26 03:00:00,1,55
2202,2019-12-08 13:30:00,1,3,2019-12-09 13:30:00,1,56,2019-12-10 13:30:00,0,7
2223,2019-12-09 11:30:00,2,2,2019-12-10 11:30:00,1,61,2019-12-11 11:30:00,0,10
2229,2019-12-09 13:00:00,1,2,2019-12-10 13:00:00,1,63,2019-12-11 13:00:00,0,0
2231,2019-12-09 13:30:00,1,2,2019-12-10 13:30:00,1,75,2019-12-11 13:30:00,0,1


In [51]:
findCondRow48(validation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
39,2019-04-05 00:00:00,2,1,2019-04-06 00:00:00,0,0,2019-04-07 00:00:00,1,67
91,2019-04-09 21:00:00,1,2,2019-04-10 21:00:00,0,3,2019-04-11 21:00:00,1,53
219,2019-04-24 16:00:00,2,1,2019-04-25 16:00:00,0,0,2019-04-26 16:00:00,1,76
220,2019-04-24 20:00:00,2,2,2019-04-25 20:00:00,0,0,2019-04-26 20:00:00,1,94
345,2019-05-07 14:00:00,1,3,2019-05-08 14:00:00,0,0,2019-05-09 14:00:00,1,59
...,...,...,...,...,...,...,...,...,...
3343,2020-03-09 23:00:00,2,1,2020-03-10 23:00:00,0,0,2020-03-11 23:00:00,1,55
3349,2020-03-10 08:30:00,2,1,2020-03-11 08:30:00,0,16,2020-03-12 08:30:00,1,69
3412,2020-03-17 17:00:00,1,3,2020-03-18 17:00:00,0,14,2020-03-19 17:00:00,1,71
3425,2020-03-19 02:30:00,1,3,2020-03-20 02:30:00,0,23,2020-03-21 02:30:00,1,70


# LightGBM
## 24 시간 결로 예측

In [52]:
from lightgbm import LGBMClassifier
lgbm_model24 = LGBMClassifier(n_estimators=100, random_state=42)
lgbm_model24.fit(X_train24, y_train24)

C:\Users\silen\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\silen\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [53]:
y_pred24 = lgbm_model24.predict(X_val24)
from sklearn.metrics import classification_report
print(classification_report(y_val24, y_pred24))

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96    104169
         1.0       0.95      0.98      0.96    104234

    accuracy                           0.96    208403
   macro avg       0.96      0.96      0.96    208403
weighted avg       0.96      0.96      0.96    208403



## 48 시간 결로 예측

In [54]:
from lightgbm import LGBMClassifier
lgbm_model48 = LGBMClassifier(n_estimators=100, random_state=42)
lgbm_model48.fit(X_train48, y_train48)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [55]:
y_pred48 = lgbm_model48.predict(X_val48)
from sklearn.metrics import classification_report
print(classification_report(y_val48, y_pred48))

              precision    recall  f1-score   support

         0.0       0.97      0.91      0.94    104163
         1.0       0.91      0.98      0.94    104240

    accuracy                           0.94    208403
   macro avg       0.94      0.94      0.94    208403
weighted avg       0.94      0.94      0.94    208403



## 검증값 만들기

In [56]:
lgValiation =makeValiation(lgbm_model24,lgbm_model48)
lgValiation

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 00:00:00,2,3,2019-04-02 00:00:00,0,0,2019-04-03 00:00:00,0,5
1,2019-04-01 04:30:00,1,2,2019-04-02 04:30:00,0,1,2019-04-03 04:30:00,0,9
2,2019-04-01 10:30:00,2,3,2019-04-02 10:30:00,0,0,2019-04-03 10:30:00,0,0
3,2019-04-01 11:00:00,1,1,2019-04-02 11:00:00,0,0,2019-04-03 11:00:00,0,0
4,2019-04-01 13:30:00,2,1,2019-04-02 13:30:00,0,0,2019-04-03 13:30:00,0,0
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 03:00:00,1,2,2020-03-30 03:00:00,0,0,2020-03-31 03:00:00,0,0
3535,2020-03-29 05:00:00,2,3,2020-03-30 05:00:00,0,0,2020-03-31 05:00:00,0,0
3536,2020-03-29 07:00:00,2,1,2020-03-30 07:00:00,0,1,2020-03-31 07:00:00,0,27
3537,2020-03-29 11:30:00,1,1,2020-03-30 11:30:00,0,0,2020-03-31 11:30:00,0,0


In [57]:
findCondRow24(lgValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
27,2019-04-03 12:30:00,1,3,2019-04-04 12:30:00,1,80,2019-04-05 12:30:00,1,69
45,2019-04-05 06:30:00,2,2,2019-04-06 06:30:00,1,89,2019-04-07 06:30:00,0,22
46,2019-04-05 08:00:00,1,3,2019-04-06 08:00:00,1,56,2019-04-07 08:00:00,1,63
111,2019-04-11 11:00:00,1,3,2019-04-12 11:00:00,1,73,2019-04-13 11:00:00,1,65
114,2019-04-11 15:30:00,2,1,2019-04-12 15:30:00,1,89,2019-04-13 15:30:00,0,5
...,...,...,...,...,...,...,...,...,...
3427,2020-03-19 04:00:00,1,2,2020-03-20 04:00:00,1,91,2020-03-21 04:00:00,1,75
3428,2020-03-19 09:30:00,2,2,2020-03-20 09:30:00,1,52,2020-03-21 09:30:00,0,47
3429,2020-03-19 10:00:00,1,3,2020-03-20 10:00:00,1,83,2020-03-21 10:00:00,1,65
3441,2020-03-21 10:30:00,1,3,2020-03-22 10:30:00,1,56,2020-03-23 10:30:00,1,76


In [58]:
findCondRow48(lgValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
21,2019-04-03 07:00:00,2,1,2019-04-04 07:00:00,0,17,2019-04-05 07:00:00,1,60
27,2019-04-03 12:30:00,1,3,2019-04-04 12:30:00,1,80,2019-04-05 12:30:00,1,69
41,2019-04-05 01:30:00,2,2,2019-04-06 01:30:00,0,2,2019-04-07 01:30:00,1,88
46,2019-04-05 08:00:00,1,3,2019-04-06 08:00:00,1,56,2019-04-07 08:00:00,1,63
64,2019-04-07 03:00:00,2,1,2019-04-08 03:00:00,0,9,2019-04-09 03:00:00,1,80
...,...,...,...,...,...,...,...,...,...
3442,2020-03-21 11:00:00,1,2,2020-03-22 11:00:00,0,46,2020-03-23 11:00:00,1,71
3480,2020-03-25 01:30:00,2,1,2020-03-26 01:30:00,0,0,2020-03-27 01:30:00,1,50
3482,2020-03-25 02:00:00,2,1,2020-03-26 02:00:00,0,0,2020-03-27 02:00:00,1,56
3490,2020-03-25 17:30:00,1,2,2020-03-26 17:30:00,0,37,2020-03-27 17:30:00,1,60


# XGBOOST
## 24 시간 결로 예측

In [59]:
from xgboost import XGBClassifier
xgb_model24 = XGBClassifier(max_depth=10, learning_rate=0.01, n_estimators=100, random_state=42)
xgb_model24.fit(X_train24, y_train24)

C:\Users\silen\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\silen\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [60]:
y_pred24 = xgb_model24.predict(X_val24)
from sklearn.metrics import classification_report
print(classification_report(y_val24, y_pred24))

              precision    recall  f1-score   support

         0.0       0.97      0.93      0.95    104169
         1.0       0.93      0.97      0.95    104234

    accuracy                           0.95    208403
   macro avg       0.95      0.95      0.95    208403
weighted avg       0.95      0.95      0.95    208403



## 48 시간 결로 예측

In [61]:
from xgboost import XGBClassifier
xgb_model48 = XGBClassifier(max_depth=10, learning_rate=0.01, n_estimators=100, random_state=42)
xgb_model48.fit(X_train48, y_train48)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [62]:
y_pred48 = xgb_model48.predict(X_val48)
from sklearn.metrics import classification_report
print(classification_report(y_val48, y_pred48))

              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93    104163
         1.0       0.89      0.98      0.93    104240

    accuracy                           0.93    208403
   macro avg       0.94      0.93      0.93    208403
weighted avg       0.94      0.93      0.93    208403



## 검증 값 만들기

In [63]:
xgValiation=makeValiation(xgb_model24,xgb_model48)
xgValiation

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 00:00:00,2,3,2019-04-02 00:00:00,0,18,2019-04-03 00:00:00,0,21
1,2019-04-01 04:30:00,1,2,2019-04-02 04:30:00,0,20,2019-04-03 04:30:00,0,22
2,2019-04-01 10:30:00,2,3,2019-04-02 10:30:00,0,18,2019-04-03 10:30:00,0,17
3,2019-04-01 11:00:00,1,1,2019-04-02 11:00:00,0,18,2019-04-03 11:00:00,0,18
4,2019-04-01 13:30:00,2,1,2019-04-02 13:30:00,0,18,2019-04-03 13:30:00,0,18
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 03:00:00,1,2,2020-03-30 03:00:00,0,18,2020-03-31 03:00:00,0,18
3535,2020-03-29 05:00:00,2,3,2020-03-30 05:00:00,0,18,2020-03-31 05:00:00,0,18
3536,2020-03-29 07:00:00,2,1,2020-03-30 07:00:00,0,18,2020-03-31 07:00:00,0,40
3537,2020-03-29 11:30:00,1,1,2020-03-30 11:30:00,0,18,2020-03-31 11:30:00,0,18


In [64]:
findCondRow24(xgValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
27,2019-04-03 12:30:00,1,3,2019-04-04 12:30:00,1,69,2019-04-05 12:30:00,1,60
44,2019-04-05 04:30:00,2,1,2019-04-06 04:30:00,1,68,2019-04-07 04:30:00,0,45
45,2019-04-05 06:30:00,2,2,2019-04-06 06:30:00,1,77,2019-04-07 06:30:00,0,48
46,2019-04-05 08:00:00,1,3,2019-04-06 08:00:00,1,73,2019-04-07 08:00:00,1,50
111,2019-04-11 11:00:00,1,3,2019-04-12 11:00:00,1,65,2019-04-13 11:00:00,1,52
...,...,...,...,...,...,...,...,...,...
3434,2020-03-20 10:00:00,1,1,2020-03-21 10:00:00,1,55,2020-03-22 10:00:00,0,18
3440,2020-03-21 07:30:00,2,1,2020-03-22 07:30:00,1,70,2020-03-23 07:30:00,1,74
3443,2020-03-21 16:00:00,2,1,2020-03-22 16:00:00,1,60,2020-03-23 16:00:00,0,18
3444,2020-03-21 21:30:00,1,3,2020-03-22 21:30:00,1,54,2020-03-23 21:30:00,0,46


In [65]:
findCondRow48(xgValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
21,2019-04-03 07:00:00,2,1,2019-04-04 07:00:00,0,28,2019-04-05 07:00:00,1,64
27,2019-04-03 12:30:00,1,3,2019-04-04 12:30:00,1,69,2019-04-05 12:30:00,1,60
39,2019-04-05 00:00:00,2,1,2019-04-06 00:00:00,0,18,2019-04-07 00:00:00,1,69
41,2019-04-05 01:30:00,2,2,2019-04-06 01:30:00,0,23,2019-04-07 01:30:00,1,74
46,2019-04-05 08:00:00,1,3,2019-04-06 08:00:00,1,73,2019-04-07 08:00:00,1,50
...,...,...,...,...,...,...,...,...,...
3440,2020-03-21 07:30:00,2,1,2020-03-22 07:30:00,1,70,2020-03-23 07:30:00,1,74
3441,2020-03-21 10:30:00,1,3,2020-03-22 10:30:00,0,43,2020-03-23 10:30:00,1,53
3442,2020-03-21 11:00:00,1,2,2020-03-22 11:00:00,0,49,2020-03-23 11:00:00,1,52
3480,2020-03-25 01:30:00,2,1,2020-03-26 01:30:00,0,18,2020-03-27 01:30:00,1,52


# KNN
## 24시간 결로 예측

In [66]:
from sklearn.neighbors import KNeighborsClassifier
neigh24 = KNeighborsClassifier(n_neighbors=3)
neigh24.fit(X_train24, y_train24)

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [67]:
y_pred24 = neigh24.predict(X_val24)
from sklearn.metrics import classification_report
print(classification_report(y_val24, y_pred24))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99    104169
         1.0       0.99      1.00      0.99    104234

    accuracy                           0.99    208403
   macro avg       0.99      0.99      0.99    208403
weighted avg       0.99      0.99      0.99    208403



## 48시간 결로 예측

In [68]:
from sklearn.neighbors import KNeighborsClassifier
neigh48 = KNeighborsClassifier(n_neighbors=3)
neigh48.fit(X_train48, y_train48)

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [69]:
y_pred48 = neigh48.predict(X_val48)
from sklearn.metrics import classification_report
print(classification_report(y_val48, y_pred48))

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99    104163
         1.0       0.98      1.00      0.99    104240

    accuracy                           0.99    208403
   macro avg       0.99      0.99      0.99    208403
weighted avg       0.99      0.99      0.99    208403



In [70]:
knnValiation=makeValiation(neigh24,neigh48)
knnValiation

C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\silen\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
0,2019-04-01 00:00:00,2,3,2019-04-02 00:00:00,0,0,2019-04-03 00:00:00,0,0
1,2019-04-01 04:30:00,1,2,2019-04-02 04:30:00,0,0,2019-04-03 04:30:00,0,0
2,2019-04-01 10:30:00,2,3,2019-04-02 10:30:00,0,0,2019-04-03 10:30:00,0,0
3,2019-04-01 11:00:00,1,1,2019-04-02 11:00:00,0,0,2019-04-03 11:00:00,0,0
4,2019-04-01 13:30:00,2,1,2019-04-02 13:30:00,0,0,2019-04-03 13:30:00,0,0
...,...,...,...,...,...,...,...,...,...
3534,2020-03-29 03:00:00,1,2,2020-03-30 03:00:00,0,0,2020-03-31 03:00:00,0,0
3535,2020-03-29 05:00:00,2,3,2020-03-30 05:00:00,0,0,2020-03-31 05:00:00,0,0
3536,2020-03-29 07:00:00,2,1,2020-03-30 07:00:00,0,0,2020-03-31 07:00:00,0,0
3537,2020-03-29 11:30:00,1,1,2020-03-30 11:30:00,0,0,2020-03-31 11:30:00,0,0


In [71]:
findCondRow24(knnValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
43,2019-04-05 02:30:00,1,3,2019-04-06 02:30:00,1,100,2019-04-07 02:30:00,0,0
412,2019-05-14 10:00:00,2,1,2019-05-15 10:00:00,1,66,2019-05-16 10:00:00,0,0
421,2019-05-15 12:30:00,2,1,2019-05-16 12:30:00,1,100,2019-05-17 12:30:00,0,0
461,2019-05-19 15:00:00,1,2,2019-05-20 15:00:00,1,100,2019-05-21 15:00:00,0,0
462,2019-05-19 16:30:00,1,3,2019-05-20 16:30:00,1,100,2019-05-21 16:30:00,0,0
...,...,...,...,...,...,...,...,...,...
3342,2020-03-09 18:30:00,2,1,2020-03-10 18:30:00,1,100,2020-03-11 18:30:00,1,100
3349,2020-03-10 08:30:00,2,1,2020-03-11 08:30:00,1,66,2020-03-12 08:30:00,1,100
3352,2020-03-10 18:00:00,1,3,2020-03-11 18:00:00,1,66,2020-03-12 18:00:00,0,0
3424,2020-03-19 01:00:00,2,2,2020-03-20 01:00:00,1,66,2020-03-21 01:00:00,0,33


In [72]:
findCondRow48(knnValiation)

,MEA_DDHR,PLANT,LOC,X24H_TMA,X24H_COND_LOC,X24H_COND_LOC_PROB,X48H_TMA,X48H_COND_LOC,X48H_COND_LOC_PROB
39,2019-04-05 00:00:00,2,1,2019-04-06 00:00:00,0,0,2019-04-07 00:00:00,1,100
41,2019-04-05 01:30:00,2,2,2019-04-06 01:30:00,0,0,2019-04-07 01:30:00,1,100
64,2019-04-07 03:00:00,2,1,2019-04-08 03:00:00,0,0,2019-04-09 03:00:00,1,100
91,2019-04-09 21:00:00,1,2,2019-04-10 21:00:00,0,0,2019-04-11 21:00:00,1,100
117,2019-04-12 03:00:00,2,1,2019-04-13 03:00:00,0,0,2019-04-14 03:00:00,1,100
...,...,...,...,...,...,...,...,...,...
3290,2020-03-04 00:30:00,1,3,2020-03-05 00:30:00,0,0,2020-03-06 00:30:00,1,66
3342,2020-03-09 18:30:00,2,1,2020-03-10 18:30:00,1,100,2020-03-11 18:30:00,1,100
3349,2020-03-10 08:30:00,2,1,2020-03-11 08:30:00,1,66,2020-03-12 08:30:00,1,100
3412,2020-03-17 17:00:00,1,3,2020-03-18 17:00:00,0,0,2020-03-19 17:00:00,1,100
